# Part 1 Data Preprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, -1].values

# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Now let's make the ANN!

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the Input Layer and the first Hidden Layer with Dropout
classifier.add(Dense(units=6, kernel_initializer="uniform", activation="relu", input_dim = 11))
classifier.add(Dropout(p=0.1))

# Adding a second Hidden Layer with Dropout
classifier.add(Dense(units=6,kernel_initializer="uniform", activation="relu"))
classifier.add(Dropout(p=0.1))

# Adding the output Layer
classifier.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

# Compiling the ANN
classifier.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train,y_train, batch_size = 10, epochs = 100)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  from ipykernel import kernelapp as app


Epoch 1/100
8000/8000 [==============================] - 1s 173us/step - loss: 0.4896 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4361 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.4338 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.4330 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.4274 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.4252 - acc: 0.8089
Epoch 7/100
8000/8000 [==============================] - 1s 98us/step - loss: 0.4272 - acc: 0.8230
Epoch 8/100
8000/8000 [==============================] - 1s 99us/step - loss: 0.4259 - acc: 0.8256
Epoch 9/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4257 - acc: 0.8265
Epoch 10/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4242 - acc: 0.8242
Epoc

# Part 3 - Making predictions and evaluating the model

In [4]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1551,   44],
       [ 280,  125]], dtype=int64)

In [10]:
cm.sum()

2000

In [11]:
accuracy = (cm[0,0] + cm[1,1]) / cm.sum()
accuracy

0.838

In [12]:
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = new_prediction > 0.5

In [13]:
new_prediction

array([[False]])

# Part 4 - Evaluating, Improving and Tuning the ANN

In [4]:
# Evaluating the ANN 
""""from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
import keras
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    
    # Initialising the ANN
    classifier = Sequential()
    
    # Adding the Input Layer and the first Hidden Layer
    classifier.add(Dense(units=6, kernel_initializer="uniform", activation="relu", input_dim = 11))

    # Adding a second Hidden Layer
    classifier.add(Dense(units=6,kernel_initializer="uniform", activation="relu"))

    # Adding the output Layer
    classifier.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

    # Compiling the ANN
    classifier.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier,X = X_train, y = y_train, cv = 10, n_jobs = -1)"""

In [4]:
# Tuning the ANN with Grid Search

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import keras
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    
    # Initialising the ANN
    classifier = Sequential()
    
    # Adding the Input Layer and the first Hidden Layer
    classifier.add(Dense(units=6, kernel_initializer="uniform", activation="relu", input_dim = 11))

    # Adding a second Hidden Layer
    classifier.add(Dense(units=6,kernel_initializer="uniform", activation="relu"))

    # Adding the output Layer
    classifier.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

    # Compiling the ANN
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)
parameters = {"batch_size":[25,32], "epochs":[100,500], "optimizer":["adam","rmsprop"]}
grid_search = GridSearchCV(estimator=classifier,param_grid=parameters,scoring="accuracy",cv=10)
grid_search = grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_Search.best_score_

Epoch 1/100
7200/7200 [==============================] - 1s 89us/step - loss: 0.5763 - acc: 0.7953
Epoch 2/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4365 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 0s 46us/step - loss: 0.4308 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.4280 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 0s 59us/step - loss: 0.4253 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 0s 50us/step - loss: 0.4227 - acc: 0.7971
Epoch 7/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4202 - acc: 0.7971
Epoch 8/100
7200/7200 [==============================] - 0s 50us/step - loss: 0.4180 - acc: 0.8110
Epoch 9/100
7200/7200 [==============================] - 0s 62us/step - loss: 0.4166 - acc: 0.8210
Epoch 10/100
7200/7200 [==============================] - 0s 46us/step - loss: 0.4154 - acc: 0.8239
Epoch 11/

7200/7200 [==============================] - 0s 40us/step - loss: 0.4005 - acc: 0.8353
Epoch 64/100
7200/7200 [==============================] - 0s 37us/step - loss: 0.4007 - acc: 0.8350
Epoch 65/100
7200/7200 [==============================] - 0s 36us/step - loss: 0.4005 - acc: 0.8369
Epoch 66/100
7200/7200 [==============================] - 0s 37us/step - loss: 0.4002 - acc: 0.8347
Epoch 67/100
7200/7200 [==============================] - 0s 37us/step - loss: 0.4004 - acc: 0.8351
Epoch 68/100
7200/7200 [==============================] - 0s 37us/step - loss: 0.4007 - acc: 0.8369
Epoch 69/100
7200/7200 [==============================] - 0s 36us/step - loss: 0.4003 - acc: 0.8365
Epoch 70/100
7200/7200 [==============================] - 0s 36us/step - loss: 0.4003 - acc: 0.8353
Epoch 71/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.4004 - acc: 0.8356
Epoch 72/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.4001 - acc: 0.8354
Epoch 73/100


Epoch 44/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3961 - acc: 0.8346
Epoch 45/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.3955 - acc: 0.8342
Epoch 46/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.3957 - acc: 0.8357
Epoch 47/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.3953 - acc: 0.8354
Epoch 48/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3950 - acc: 0.8357
Epoch 49/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3953 - acc: 0.8360
Epoch 50/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3948 - acc: 0.8343
Epoch 51/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3947 - acc: 0.8358
Epoch 52/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3944 - acc: 0.8369
Epoch 53/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3941 - acc: 0.8361


7200/7200 [==============================] - 0s 40us/step - loss: 0.4234 - acc: 0.8036
Epoch 7/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4214 - acc: 0.8183
Epoch 8/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4204 - acc: 0.8229: 0s - loss: 0.4258 - acc: 0.82
Epoch 9/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4193 - acc: 0.8253
Epoch 10/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4178 - acc: 0.8260
Epoch 11/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4168 - acc: 0.8276
Epoch 12/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4164 - acc: 0.8304
Epoch 13/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4152 - acc: 0.8303
Epoch 14/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4144 - acc: 0.8318
Epoch 15/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4137

7200/7200 [==============================] - 0s 41us/step - loss: 0.4009 - acc: 0.8350
Epoch 68/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4008 - acc: 0.8335
Epoch 69/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4005 - acc: 0.8339: 0s - loss: 0.3963 - acc: 0.836
Epoch 70/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4005 - acc: 0.8347
Epoch 71/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4007 - acc: 0.8336
Epoch 72/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4008 - acc: 0.8356
Epoch 73/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4004 - acc: 0.8339
Epoch 74/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4003 - acc: 0.8343
Epoch 75/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4002 - acc: 0.8354
Epoch 76/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.

...

7200/7200 [==============================] - 0s 68us/step - loss: 0.4005 - acc: 0.8353
Epoch 476/500
7200/7200 [==============================] - 0s 69us/step - loss: 0.4006 - acc: 0.8343
Epoch 477/500
7200/7200 [==============================] - 1s 79us/step - loss: 0.4002 - acc: 0.8382
Epoch 478/500
7200/7200 [==============================] - 1s 90us/step - loss: 0.4007 - acc: 0.8356
Epoch 479/500
7200/7200 [==============================] - 1s 74us/step - loss: 0.4004 - acc: 0.8353
Epoch 480/500
7200/7200 [==============================] - 0s 69us/step - loss: 0.4004 - acc: 0.8351
Epoch 481/500
7200/7200 [==============================] - 0s 69us/step - loss: 0.4004 - acc: 0.8339
Epoch 482/500
7200/7200 [==============================] - 0s 68us/step - loss: 0.4006 - acc: 0.8358
Epoch 483/500
7200/7200 [==============================] - 0s 69us/step - loss: 0.4006 - acc: 0.8336
Epoch 484/500
7200/7200 [==============================] - 0s 69us/step - loss: 0.4002 - acc: 0.8349
Epoc

Epoch 55/500
8000/8000 [==============================] - 1s 88us/step - loss: 0.4013 - acc: 0.8350
Epoch 56/500
8000/8000 [==============================] - 1s 87us/step - loss: 0.4012 - acc: 0.8347
Epoch 57/500
8000/8000 [==============================] - 1s 88us/step - loss: 0.4009 - acc: 0.8350
Epoch 58/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.4008 - acc: 0.8340
Epoch 59/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.4005 - acc: 0.8341
Epoch 60/500
8000/8000 [==============================] - 1s 88us/step - loss: 0.4009 - acc: 0.8339
Epoch 61/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.4008 - acc: 0.8342: 0s - loss: 0.3958 - acc
Epoch 62/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.4007 - acc: 0.8344
Epoch 63/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.4007 - acc: 0.8350
Epoch 64/500
8000/8000 [==============================] - 1s 88us/step - lo

8000/8000 [==============================] - 1s 99us/step - loss: 0.3993 - acc: 0.8357
Epoch 213/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3994 - acc: 0.8355
Epoch 214/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3993 - acc: 0.8364
Epoch 215/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3991 - acc: 0.8357
Epoch 216/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3993 - acc: 0.8361
Epoch 217/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3993 - acc: 0.8357
Epoch 218/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3994 - acc: 0.8356
Epoch 219/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3992 - acc: 0.8361
Epoch 220/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3992 - acc: 0.8361
Epoch 221/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3993 - acc: 0.8334
Epoc

8000/8000 [==============================] - 1s 97us/step - loss: 0.3988 - acc: 0.8342
Epoch 373/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3990 - acc: 0.8367
Epoch 374/500
8000/8000 [==============================] - 1s 99us/step - loss: 0.3991 - acc: 0.8359
Epoch 375/500
8000/8000 [==============================] - 1s 98us/step - loss: 0.3989 - acc: 0.8360
Epoch 376/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3992 - acc: 0.8364
Epoch 377/500
8000/8000 [==============================] - 1s 98us/step - loss: 0.3989 - acc: 0.8342
Epoch 378/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3991 - acc: 0.8355
Epoch 379/500
8000/8000 [==============================] - 1s 99us/step - loss: 0.3987 - acc: 0.8360
Epoch 380/500
8000/8000 [==============================] - 1s 103us/step - loss: 0.3990 - acc: 0.8357
Epoch 381/500
8000/8000 [==============================] - 1s 109us/step - loss: 0.3989 - acc: 0.8346
Ep

NameError: name 'grid_Search' is not defined

In [5]:
best_parameters

{'batch_size': 25, 'epochs': 500, 'optimizer': 'adam'}

In [6]:
best_accuracy = grid_search.best_score_
best_accuracy

0.850875

# Making the Model again with Tuned Parameters

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the Input Layer and the first Hidden Layer with Dropout
classifier.add(Dense(units=6, kernel_initializer="uniform", activation="relu", input_dim = 11))
classifier.add(Dropout(p=0.1))

# Adding a second Hidden Layer with Dropout
classifier.add(Dense(units=6,kernel_initializer="uniform", activation="relu"))
classifier.add(Dropout(p=0.1))

# Adding the output Layer
classifier.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

# Compiling the ANN
classifier.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train,y_train, batch_size = 25, epochs = 500)

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  from ipykernel import kernelapp as app


Epoch 1/500
8000/8000 [==============================] - 2s 192us/step - loss: 0.5604 - acc: 0.7949
Epoch 2/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.4437 - acc: 0.7960
Epoch 3/500
8000/8000 [==============================] - 1s 81us/step - loss: 0.4371 - acc: 0.7960
Epoch 4/500
8000/8000 [==============================] - 1s 90us/step - loss: 0.4362 - acc: 0.7960
Epoch 5/500
8000/8000 [==============================] - 1s 72us/step - loss: 0.4355 - acc: 0.7960
Epoch 6/500
8000/8000 [==============================] - 1s 72us/step - loss: 0.4355 - acc: 0.7960
Epoch 7/500
8000/8000 [==============================] - 1s 70us/step - loss: 0.4332 - acc: 0.7960
Epoch 8/500
8000/8000 [==============================] - 1s 71us/step - loss: 0.4276 - acc: 0.7960
Epoch 9/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.4278 - acc: 0.7960
Epoch 10/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.4305 - acc: 0.7960
Epoch 11

8000/8000 [==============================] - 1s 63us/step - loss: 0.4188 - acc: 0.8311
Epoch 82/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.4217 - acc: 0.8311
Epoch 83/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.4213 - acc: 0.8302
Epoch 84/500
8000/8000 [==============================] - 1s 89us/step - loss: 0.4201 - acc: 0.8305
Epoch 85/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.4219 - acc: 0.8304
Epoch 86/500
8000/8000 [==============================] - 1s 67us/step - loss: 0.4218 - acc: 0.8296
Epoch 87/500
8000/8000 [==============================] - 1s 70us/step - loss: 0.4189 - acc: 0.8316
Epoch 88/500
8000/8000 [==============================] - 1s 63us/step - loss: 0.4203 - acc: 0.8327
Epoch 89/500
8000/8000 [==============================] - 0s 60us/step - loss: 0.4201 - acc: 0.8306
Epoch 90/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.4196 - acc: 0.8329
Epoch 91/500


8000/8000 [==============================] - 1s 68us/step - loss: 0.4240 - acc: 0.8326
Epoch 162/500
8000/8000 [==============================] - 1s 75us/step - loss: 0.4238 - acc: 0.8322
Epoch 163/500
8000/8000 [==============================] - 0s 62us/step - loss: 0.4230 - acc: 0.8291
Epoch 164/500
8000/8000 [==============================] - 1s 67us/step - loss: 0.4211 - acc: 0.8310
Epoch 165/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.4187 - acc: 0.8324
Epoch 166/500
8000/8000 [==============================] - 1s 75us/step - loss: 0.4190 - acc: 0.8327
Epoch 167/500
8000/8000 [==============================] - 1s 63us/step - loss: 0.4232 - acc: 0.8334
Epoch 168/500
8000/8000 [==============================] - 1s 71us/step - loss: 0.4210 - acc: 0.8310
Epoch 169/500
8000/8000 [==============================] - 1s 65us/step - loss: 0.4196 - acc: 0.8324
Epoch 170/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.4196 - acc: 0.8311
Epoc

8000/8000 [==============================] - 1s 81us/step - loss: 0.4189 - acc: 0.8341
Epoch 241/500
8000/8000 [==============================] - 1s 67us/step - loss: 0.4214 - acc: 0.8336
Epoch 242/500
8000/8000 [==============================] - 1s 77us/step - loss: 0.4194 - acc: 0.8326
Epoch 243/500
8000/8000 [==============================] - 1s 75us/step - loss: 0.4189 - acc: 0.8301
Epoch 244/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.4207 - acc: 0.8317
Epoch 245/500
8000/8000 [==============================] - 1s 75us/step - loss: 0.4170 - acc: 0.8341
Epoch 246/500
8000/8000 [==============================] - 1s 98us/step - loss: 0.4195 - acc: 0.8337
Epoch 247/500
8000/8000 [==============================] - 1s 82us/step - loss: 0.4180 - acc: 0.8342: 0s - loss: 0.4118 - ac
Epoch 248/500
8000/8000 [==============================] - 1s 79us/step - loss: 0.4225 - acc: 0.8301
Epoch 249/500
8000/8000 [==============================] - 1s 85us/step - loss: 0

8000/8000 [==============================] - 1s 75us/step - loss: 0.4225 - acc: 0.8340
Epoch 321/500
8000/8000 [==============================] - 1s 71us/step - loss: 0.4210 - acc: 0.8312
Epoch 322/500
8000/8000 [==============================] - 1s 67us/step - loss: 0.4196 - acc: 0.8321
Epoch 323/500
8000/8000 [==============================] - 1s 67us/step - loss: 0.4176 - acc: 0.8326
Epoch 324/500
8000/8000 [==============================] - 1s 71us/step - loss: 0.4221 - acc: 0.8302
Epoch 325/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.4191 - acc: 0.8335
Epoch 326/500
8000/8000 [==============================] - 1s 69us/step - loss: 0.4203 - acc: 0.8329
Epoch 327/500
8000/8000 [==============================] - 1s 70us/step - loss: 0.4192 - acc: 0.8319
Epoch 328/500
8000/8000 [==============================] - 1s 100us/step - loss: 0.4196 - acc: 0.8349
Epoch 329/500
8000/8000 [==============================] - 1s 85us/step - loss: 0.4183 - acc: 0.8337
Epo

8000/8000 [==============================] - 1s 65us/step - loss: 0.4216 - acc: 0.8330
Epoch 401/500
8000/8000 [==============================] - 0s 62us/step - loss: 0.4194 - acc: 0.8342
Epoch 402/500
8000/8000 [==============================] - 1s 64us/step - loss: 0.4185 - acc: 0.8332
Epoch 403/500
8000/8000 [==============================] - 1s 70us/step - loss: 0.4167 - acc: 0.8304
Epoch 404/500
8000/8000 [==============================] - 1s 65us/step - loss: 0.4199 - acc: 0.8319
Epoch 405/500
8000/8000 [==============================] - 1s 66us/step - loss: 0.4176 - acc: 0.8324
Epoch 406/500
8000/8000 [==============================] - 1s 66us/step - loss: 0.4204 - acc: 0.8337
Epoch 407/500
8000/8000 [==============================] - 1s 68us/step - loss: 0.4186 - acc: 0.8336
Epoch 408/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.4193 - acc: 0.8330
Epoch 409/500
8000/8000 [==============================] - 1s 63us/step - loss: 0.4196 - acc: 0.8324
Epoc

8000/8000 [==============================] - 1s 70us/step - loss: 0.4172 - acc: 0.8309
Epoch 481/500
8000/8000 [==============================] - 1s 67us/step - loss: 0.4208 - acc: 0.8327
Epoch 482/500
8000/8000 [==============================] - 1s 76us/step - loss: 0.4192 - acc: 0.8372
Epoch 483/500
8000/8000 [==============================] - 1s 69us/step - loss: 0.4176 - acc: 0.8331
Epoch 484/500
8000/8000 [==============================] - 1s 74us/step - loss: 0.4184 - acc: 0.8330
Epoch 485/500
8000/8000 [==============================] - 1s 69us/step - loss: 0.4162 - acc: 0.8310
Epoch 486/500
8000/8000 [==============================] - 1s 70us/step - loss: 0.4198 - acc: 0.8345
Epoch 487/500
8000/8000 [==============================] - 0s 61us/step - loss: 0.4209 - acc: 0.8320
Epoch 488/500
8000/8000 [==============================] - 1s 73us/step - loss: 0.4214 - acc: 0.8325
Epoch 489/500
8000/8000 [==============================] - 1s 86us/step - loss: 0.4190 - acc: 0.8322
Epoc

In [3]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1551,   44],
       [ 275,  130]], dtype=int64)

In [4]:
accuracy = (cm[0,0] + cm[1,1]) / cm.sum()
accuracy

0.8405

In [5]:
classifier.save('ANN.h5')

In [6]:
from keras.models import load_model
pretrained_model = load_model('ANN.h5')

In [7]:
pretrained_model

In [9]:
new_prediction = pretrained_model.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = new_prediction > 0.5

In [10]:
new_prediction

array([[False]])